###### Library DB - author, book
###### author - id, name, book
###### book - id, title, author_id(f key)
###### one author can write many books

In [1]:
from flask  import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

## Instatantiate the app

In [2]:
app=Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///library.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS']=False
db = SQLAlchemy(app)
# db.init_app(app)


In [3]:
class Author(db.Model):
   id=db.Column(db.Integer, primary_key=True)
   name=db.Column(db.String(50),nullable=False) 
   books=db.relationship('Book',backref='author',lazy=True)

In [4]:
class Book(db.Model):
    id=db.Column(db.Integer, primary_key=True)
    title=db.Column(db.String(50),nullable=False) 
    author=db.Column(db.Integer,db.ForeignKey('author.id'),nullable=False) 
    
    

### create table 

In [5]:
with app.app_context():
    db.create_all()

### Home page

In [6]:
@app.route('/')
def index():
    authors = Author.query.all()
    return render_template('index_library.html',authors)

### add author

In [7]:
@app.route("/add_author", methods=['GET','POST'])
def add_author():
    if request.method == 'POST':
        author_name = request.form['author']
        new_author=Author(name=author_name)
        db.session.add(new_author)
        db.session.commit()
    return redirect(url_for('index'))
        

### add books

In [8]:
@app.route("/add_book/<int:author_id>", methods=['GET','POST'])
def add_book(author_id):
    if request.method == 'POST':
        book_title = request.form['book_title']
        author=Author.query.get(author_id)
        if author:
            new_book = Book(title=book_title,author=author)
            db.session.add(new_book)
            db.session.commit()
    return redirect(url_for('index'))
        

In [9]:
app.run(port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
[2024-05-13 12:37:44,333] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\shrik\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shrik\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shrik\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shrik\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
